In [ ]:
%load_ext autoreload
%autoreload 2

# Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from skportfolio.datasets import load_dataset,get_dataset_names

## Theming and more beautiful charts

In [ ]:
# pip install ing-theme-matplotlib

In [ ]:
%config InlineBackend.figure_format = 'svg' # makes the plots HD in the notebook
mpl.rcParams["figure.autolayout"] = True # enables tigh layout. Better multiplots
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'

# from ing_theme_matplotlib import mpl_style
# mpl_style(dark=True)

## Explore some of the `scikit-portfolio` datasets.
They are stored in the `skportfolio-data` repository

In [ ]:
get_dataset_names()

# Load tech stock prices over 4 years

In [ ]:
prices = load_dataset('tech_stocks')

# First basic portfolio optimization with Markowitz efficient frontier

Here we calculate the optimal portfolio using Mean-Variance optimization.

In [ ]:
from skportfolio import MeanVarianceEfficientReturn

In [ ]:
MeanVarianceEfficientReturn().fit(prices).weights_

We can also plot the entire efficient frontier, together with the point where the current portfolio lies

In [ ]:
from skportfolio.plotting import plot_frontier
plot_frontier(
    MeanVarianceEfficientReturn(target_return=0.2).fit(load_dataset('crypto_large')),
    prices,
)
plt.grid(True)

We can plot multiple portfolios over the same efficient frontier

In [ ]:
from skportfolio.plotting import plot_frontier
fig, ax = plt.subplots(figsize=(9,6))
plot_frontier(
    MeanVarianceEfficientReturn().set_target_return(0.55).fit(prices),
    prices,
    num_portfolios=100,
    ax=ax,
    show_assets=True
)

plot_frontier(
    MeanVarianceEfficientReturn().set_target_return(0.65).fit(prices),
    prices,
    num_portfolios=0,
    ax=ax,
    show_only_portfolio=True,
    risk_return_color='indianred'
)
ax.grid(True)
ax.set_yticks(np.arange(0.35,0.75,0.05));

You can add to this visualization the Maximum Sharpe ratio portfolio. Here, importantly one has to acknowledge that the expected returns will be probably far off their "true" value, as a linear expectation is used to compute them.

In [ ]:
from skportfolio.frontier import MaxSharpe

ax = plot_frontier(
    MaxSharpe(risk_free_rate=0.1).fit(prices),
    prices,
    num_portfolios=128,
    risk_return_color='C3',
    show_tangency_line=True,
    tangency_line_color="C3",
    figsize=(9,5)
)

# Compare two efficient frontiers: which method dominates?

The point identified given a risk free rate is the tangency portfolio with the risk_free_rate of 0.1.

You can also visualize the effects of changing the expected returns calculator, and replot a new efficient frontier.

In [ ]:
from skportfolio.riskreturn import MeanHistoricalLinearReturns, CAPMReturns

ax = plot_frontier(
    MaxSharpe(risk_free_rate=0.1).fit(prices),
    prices,
    num_portfolios=20,
    frontier_line_color='C2',
    risk_return_color='C2',
    figsize=(9,5),
)

plot_frontier(
    MaxSharpe(rets_estimator=CAPMReturns()).fit(prices),
    prices,
    num_portfolios=50,
    risk_return_color='C0',
    frontier_line_color='C0',
    ax=ax
)
ax.grid(True)

But what if instead of we want to experiment the effects on the efficient frontier of different estimators of the Covariance?
We know that MVO framework is highly sensible first to the expected returns, but also to the covariance. How do we keep in to account the noise in the estimation? We can first try with the famous **Ledoit-Wolf** regularization.

In [ ]:
from skportfolio.riskreturn import CovarianceLedoitWolf, CovarianceGlasso, CovarianceExp,SampleCovariance

ax = plot_frontier(
    MaxSharpe(risk_estimator=SampleCovariance()).fit(prices),
    prices,
    num_portfolios=50,
    risk_return_color='C0',
    frontier_line_color='C0',
    figsize=(9,5)
)

plot_frontier(
    MaxSharpe(risk_estimator=CovarianceLedoitWolf()).fit(prices),
    prices,
    num_portfolios=50,
    risk_return_color='C1',
    frontier_line_color='C1',
    ax=ax,
    autoset_lims=False
)


plot_frontier(
    MaxSharpe(risk_estimator=CovarianceExp(span=180)).fit(prices),
    prices,
    num_portfolios=50,
    risk_return_color='C2',
    frontier_line_color='C2',
    ax=ax,
    autoset_lims=False
)
ax.set_xlim([0.2,0.6])

Then jump to more complicated methods, like the GLASSO method

## Accessing the efficient frontier points

You can not only **plot** the efficient frontier, but every frontier portfolio object has the `.estimate_frontier` method.
The method returns a triplet where the first two elements represent, the risk and return coordinates of the portfolios along the efficient frontier.
The last element contains the portfolio weights along each point.
Portfolios are indexed from the least risky to the maximum return along the frontier.

In [ ]:
from skportfolio.frontier import MinimumVolatility

MinimumVolatility().estimate_frontier(prices,num_portfolios=32)

## Adding other constraints to portfolio estimators
You can add other constraints to the portfolio estimator objects, such as sector constraints.
For the moment you need to indicate the asset with integer notation for the constraints.

In [ ]:
m = MinimumVolatility()
m.add_constraints([lambda w: w[4]>=0.4])

In [ ]:
m.fit(prices).weights_

### Setting more complex constraints
More complex constraints can be enforced though!
Here we set the sum of `AAPL` and `TSLA` to be less than 20% of entire portfolio!

In [ ]:
m = MinimumVolatility()
m.add_constraints([lambda w: (w[0]+w[1]) <= 0.2])
print(m.fit(prices).weights_.round(4).mul(100))

from skportfolio.plotting import pie_chart
pie_chart(m.weights_, color='black')

Remember that, for the moment, long-only portfolios are allowed, hence weights must sum to 1 and be positive!

# How to bootstrap scenarios on expected returns?
## Michaud Resampled Efficient Frontier

The estimation of expected returns has a large effect on the calculation of the efficient frontier.
For this reason, one approach is based on bootstraping of the expected returns.
The idea of the Michaud Resampled Efficient frontier is encoded through the `PerturbedLinearEstimator`.

Every time you run it you sample a neighborhood of the efficient frontier, and you become able to smooth the extreme allocations provided by the Markowitz framework (this indeed works for all possible portfolio estimators requiring the expected returns estimator).

In [ ]:
from skportfolio.riskreturn import PerturbedReturns
m = MaxSharpe(rets_estimator=PerturbedReturns()).fit(prices)
m.weights_

In [ ]:
m.fit(prices).weights_

In this case a way to estimate the resampled frontier is through the Michaud algorithm.
Here we resample **1024 independent scenarios** around the efficient return MVO portfolio, using parallelization.

Mathematically, at each scenario we sample the returns $\tilde{r} \sim \textrm{N}\left( \boldsymbol \mu , \hat{\boldsymbol \Sigma}) \right)$. Each scenario being independent, produces a new value of the empirical expected returns $\boldsymbol \mu$. The sample covariance is kept intact, though.

The meta-estimator `MichaudResampledFrontier` implements this idea.

In [ ]:
from skportfolio.ensemble import MichaudResampledFrontier

resampled_ptf = MichaudResampledFrontier(
    ptf_estimator=MeanVarianceEfficientReturn(target_return=0.55),
    n_jobs=16,
    n_iter=1024
).fit(prices)

This procedures produces `n_iter` efficient frontiers, and we can average over all the portfolio weights at each point in the frontier.
In case of excessive perturbation of the resampled expected returns, the problem may be infeasible. In those cases we ignore the scenario, that's why the effective number or scenarios may be lower than `n_iter`.

In [ ]:
len(resampled_ptf.all_weights_)

Here all portfolios have been generated in parallel, thanks to `joblib`.

A Box plot makes it possible to visualize the best allocation while also removing outlier allocations resulting from extreme weights.

In [ ]:
sns.violinplot(
    data=pd.concat(resampled_ptf.all_weights_,axis=1).T.reset_index(drop=True),cut=0
)
plt.ylim([0,1])
plt.yticks(np.arange(0,1,0.1))
plt.grid()

The MichaudResampledFrontier portfolio is the following. 
As one can easly observe, by averaging over many portfolios, it tends to smooth out large too skewed allocations.

In [ ]:
fig, ax=plt.subplots(ncols=2,figsize=(10,6))
pie_chart(MeanVarianceEfficientReturn(target_return=0.55).fit(prices).weights_, ax=ax[0])
pie_chart(resampled_ptf.weights_, ax=ax[1])
plt.tight_layout()

# Alternative ensemble methods: SubsetResampling

The technique works by averaging allocation estimated with a noise-sensitive portfolio estimator (such as `MaxSharpeRatio`) over many *masked* subsets of assets.

The mask-optimization process is repeated many times: every iteration considers only a certain fraction of the entire asset universe, and optimal weights are computed.
All the local estimates are then aggregated either by averaging or by voting mechanism, and a final allocation is 
produced.

Here we use another dataset, based on the prices of 74 cryptocurrencies, and the `MinimumVolatility` portfolio.

In [ ]:
from skportfolio.ensemble import SubsetResampling
# fit on price data and average over 64 estimates, 
# considering ceil(5**0.5) = 3 stocks per subsample
subset_ptf = SubsetResampling(
	ptf_estimator=MinimumVolatility(
        risk_estimator=CovarianceLedoitWolf()
    ),
	subset_size=0.8,
	n_iter=1024,
	n_jobs=16,
	agg_func="mean"
).fit(load_dataset('crypto_large'))

In [ ]:
pie_chart(subset_ptf.weights_, threshold=8)

## Visualization of allocation as a function of portfolio risk

It is easy to generate a nice plot of portfolio allocation as a function of its risk. This function works with **any** portfolio. It additionally prints a vertical line to indicate the current portfolio, if already fitted to data.

In [ ]:
from skportfolio.frontier import MeanVarianceEfficientRisk
from skportfolio.plotting import risk_allocation_chart

Divide the dataset in train and test:

In [ ]:
from sklearn.model_selection import train_test_split
prices_train, prices_test = train_test_split(prices,shuffle=False, test_size=0.3)

In [ ]:
fig, ax = plt.subplots(ncols=2,figsize=(10,5))
risk_allocation_chart(MeanVarianceEfficientRisk(target_risk=0.3).fit(prices_train), prices_train,ax=ax[0])
risk_allocation_chart(MeanVarianceEfficientRisk(target_risk=0.3).fit(prices_train), prices_test, ax=ax[1])

ax[0].set_title('Allocation on train data')
ax[1].set_title('Same method results \n on different allocation on test data')

ax[1].set_xlim(ax[0].get_xlim());

This plots tells us the allocations on train data, once applied to out-of-sample data, may result in impossible satisfaction of constraints. For this reason we must try to optimize the parameters of portfolio methods on many **folds**.

# Cross-validation and backtesting in `scikit-portfolio`

The function `portfolio_cross_validate` is very similar to `sklearn.model_selection.cross_validate` but with a few modifications to work in `scikit-portfolio`.
The signature is the same though, but the output is always in the form of a `pd.DataFrame`.

In [ ]:
from skportfolio.metrics import sharpe_ratio
MinimumVolatility(frequency=1).fit(prices).score(
    prices, score_function=lambda x: sharpe_ratio(x,frequency=1, riskfree_rate=0)
)

In [ ]:
dow = load_dataset('dowportfolio')
dow

In [ ]:
from skportfolio.metrics import backtest
portfolios = [
    MinimumVolatility(),
    MinimumVolatility(risk_estimator=CovarianceExp(span=180)),
    MaxSharpe(),
    MaxSharpe(l2_gamma=0.1),
    MeanVarianceEfficientReturn(target_return=0.55)
]

backtest(
    dow, 
    weights=[
        m.fit(dow).weights_ for m in portfolios
    ]
).T.style.background_gradient(cmap='RdYlGn',axis=0)

Here we define a simple function to show the portfolio allocation

In [ ]:
from typing import Iterable
def portfolio_allocation_chart(weights: Iterable[pd.Series]):
    sns.heatmap(
        pd.concat((w for w in weights),axis=1).mul(100).round(1)
    )

portfolio_allocation_chart(
    [
        m.fit(dow).weights_ for m in portfolios
    ]
)

`scikit-portfolio` offers a way to perform cross-validation of metrics over a number of folds. It's very similar to `sklearn.model_selection.cross_validate`.

In [ ]:
from skportfolio.model_selection import portfolio_cross_validate,KFold
from sklearn.model_selection import LeavePOut
from skportfolio.metrics import all_scorers # all the scorers obtained from the metrics (they encode automatically if greater or smaller is better)

cv_results = portfolio_cross_validate(
    estimator=portfolios,
    prices_or_returns=dow,
    cv=KFold(10),
    scoring=all_scorers,
    n_jobs=8,
)

P.S. To be plotted meaningfully in seaborn here some reshaping of the table is in order

In [ ]:
cv_results_reshaped = (
    cv_results
    .filter(like='test_', axis=1)
    .reset_index()
    .stack()
    .drop('fold',axis=1)
    .stack()
    .droplevel(0, axis=0)
    .swaplevel()
    .reset_index()
    .rename(columns={'level_0':'variable','level_1':'variable',0:'value'})
)
cv_results_reshaped

Here the Box and Whiskers plot is done over the number of folds in the `cv_results`, hence 12 folds.

In [ ]:
sns.catplot(
    data=cv_results_reshaped,
    kind='strip',
    col='estimator',
    row='variable',
    hue='variable',
    #orient='vertical',
    height=2.5,
    aspect=1.5,
    y='value',
    sharey=False,
    margin_titles=True,
    facet_kws={'despine':True}
)
plt.subplots_adjust()
plt.tight_layout()

# Hyperparameter optimization in `scikit-portfolio`

One of the great things about this library is the possibility to perform cross-validation based hyperparameters optimization!

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split

dow_train, dow_test = train_test_split(dow, test_size=0.2, shuffle=False)
# define a base portfolio estimator whose parameters we want to estimate
baseline_ptf = MinimumVolatility()

## Instead of grid-search we use RandomizedSearchCV for faster exploration of hyperparameters

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, loguniform

rand_cv_ptf = RandomizedSearchCV(
    estimator = baseline_ptf,
    n_iter=512,
    cv=KFold(4),
    param_distributions=dict(l2_gamma=loguniform(a=10**-5, b=10**2)),
    scoring = all_scorers['omega_ratio'],
    refit=True,
    n_jobs=15
).fit(dow_train)

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
plot_frontier(
    baseline_ptf,
    dow_train,
    frontier_line_color="C0",
    ax=ax
)

plot_frontier(
    ptf_estimator=rand_cv_ptf.best_estimator_,
    prices_or_returns=dow_train,
    num_portfolios=100,
    frontier_line_color="C2",
    risk_return_color="C2",
    ax=ax
)
ax.grid(True)

Then the best estimator can be found and compared with the baseline.

In [ ]:
from skportfolio.metrics import equity_curve

In [ ]:
from skportfolio import EquallyWeighted

fig, ax = plt.subplots(ncols=2,figsize=(14,5))
##################################################
equity_curve(baseline_ptf.fit(dow_train).predict(dow_train)).plot(label='baseline',ax=ax[0])
equity_curve(rand_cv_ptf.best_estimator_.predict(dow_train)).plot(label='optimized',ax=ax[0])
equity_curve(EquallyWeighted().fit(dow_train).predict(dow_train)).plot(label='EW',ax=ax[0])
ax[0].set_title('Train set performance')
ax[0].legend()

equity_curve(baseline_ptf.fit(dow_train).predict(dow_test)).plot(label='baseline',ax=ax[1])
equity_curve(rand_cv_ptf.best_estimator_.predict(dow_test)).plot(label='optimized',ax=ax[1])
equity_curve(EquallyWeighted().fit(dow_train).predict(dow_test)).plot(label='EW',ax=ax[1])
ax[1].legend()
ax[1].set_title('Out-of-sample performance')

ax[0].grid()
ax[1].grid()
plt.subplots_adjust()
plt.tight_layout()

But to make sure, looking at prices is not the best way to compare performances of portfolios. You should create a tearsheet.

In [ ]:
pd.concat([
        baseline_ptf.weights_,
        rand_cv_ptf.best_estimator_.weights_,
        EquallyWeighted().fit(dow_train).weights_        
    ],axis=1
)

In [ ]:
from skportfolio.metrics import backtest

bt = backtest(
    prices=dow_train,
    weights=[
        baseline_ptf.weights_,
        #rand_cv_ptf.best_estimator_.weights_,
        #EquallyWeighted().fit(dow_train).weights_        
    ],
    frequency=1
)

# Portfolio optimization with the Omega ratio

Here is an illustration of the Omega ratio: the ratio between the green and the red areas in the cumulative returns distribution.

![](https://scikit-portfolio.github.io/scikit-portfolio/imgs/omega_ratio.svg)

Like the `OmegaRatio` frontier we try to maximize the OmegaRatio directly, and additional hyperparameters minimizing the annual volatility!

At the denominator we have the **red** area (omega risk, x-axis) while at the numerator the **green** area (reward).

In [ ]:
# in setting the minimum acceptable return for omega ratio portfolio, you could start from the median of the mean returns
mar = dow_train.pct_change().mean().mean()

In [ ]:
from skportfolio.frontier import MaxOmegaRatio
plt.style.use('default')
plot_frontier(
    MaxOmegaRatio(minimum_acceptable_return=mar).fit(dow),
    dow,
    #show_assets=True
)
plt.grid(False)

P.S. Here the minimim acceptable return is in daily terms!

In [ ]:
rand_cv_omega_ptf = RandomizedSearchCV(
    estimator = MaxOmegaRatio(),
    n_iter=512,
    cv=KFold(4),
    param_distributions=dict(minimum_acceptable_return=uniform(mar*0.8, mar*1.5)),
    scoring = all_scorers['omega_ratio'],
    refit=True,
    n_jobs=15
).fit(dow_train)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
equity_curve(MaxOmegaRatio(minimum_acceptable_return=mar).fit(dow_train).predict(dow_train)).plot()
equity_curve(rand_cv_omega_ptf.predict(dow_train)).plot()
equity_curve(MaxSharpe().fit(dow_train).predict(dow_train)).plot()
equity_curve(MinimumVolatility().fit(dow_train).predict(dow_train)).plot()
plt.legend()
plt.grid()

Interestingly, the MaxOmega ratio itself has an efficient frontier that you can plot! You are not limited to Markowitz frontier.

In [ ]:
plot_frontier(
    ptf_estimator=rand_cv_omega_ptf.best_estimator_,
    prices_or_returns=dow_train
)

# Mean Absolute Deviation portfolio

Also the MeanAbsoluteDeviation frontier is available!
You can see how it compares with the minimum volatility portfolio, in fact they are pretty similar.

This is a small comparison in terms of equity curves for different portfolio strategies.
There is no rebalancing here.

In [ ]:
from skportfolio import MinimumMAD

fig, ax = plt.subplots(figsize=(12,5))
equity_curve(MaxOmegaRatio(minimum_acceptable_return=mar).fit(dow_train).predict(dow_train)).plot()
equity_curve(rand_cv_omega_ptf.predict(dow_train)).plot()
equity_curve(MaxSharpe().fit(dow_train).predict(dow_train)).plot()
equity_curve(MinimumVolatility().fit(dow_train).predict(dow_train)).plot()
equity_curve(MinimumMAD().fit(dow_train).predict(dow_train)).plot()
plt.legend()
plt.grid()

Also for MeanAbsoluteDeviation portfolio you can display the efficient frontier. In this case on the x-axis you have the *Mean Absolute Deviation*, i.e. the following quantity:

$$
\mathbb{E} \left \lbrack \left| (\mathbf{R}  - \boldsymbol \mu)^T \mathbf{w} \right| \right\rbrack = \frac{1}{T} \sum_{t=1}^T \sum_{i=1}^N \bigl| (R_{t,i} - \mu_i) w_i \bigr|
$$

which is another measure of portfolio volatility.

In [ ]:
from skportfolio import MADEfficientReturn
ax = plot_frontier(
    MinimumMAD().fit(dow_train),
    dow_train,
    frontier_line_color='green',
    risk_return_color='green'
)

plot_frontier(
    MADEfficientReturn(target_return=0.0022).fit(dow_train),
    dow_train,
    frontier_line_color='indianred',
    show_assets=True,
    ax=ax
)
ax.grid(True)

# Exploring different risk estimators through cross-validation

In [ ]:
from skportfolio.riskreturn import all_risk_estimators
all_risk_estimators

In [ ]:
min_vol_cv = GridSearchCV(
    estimator = MinimumVolatility(),
    param_grid={
        'risk_estimator':
        all_risk_estimators
    },
    scoring = all_scorers['sharpe_ratio'],
    cv=KFold(10)
).fit(dow_train)

In [ ]:
backtest(
    dow_test,
    weights=[MinimumVolatility().fit(dow_train).weights_, min_vol_cv.best_estimator_.weights_]
)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
equity_curve(MinimumVolatility().fit(dow_train).predict(dow_test)).plot(ax=ax)
equity_curve(min_vol_cv.predict(dow_test)).plot(ax=ax)
plt.grid()
plt.legend()

# Hyperparameters optimization with Optuna

In [ ]:
dow.pct_change().max().max()

In [ ]:
import warnings
from optuna.integration import OptunaSearchCV
from optuna.distributions import BaseDistribution
from optuna.distributions import CategoricalDistribution
from optuna.distributions import LogUniformDistribution
from optuna.distributions import UniformDistribution
from skportfolio.metrics import *

optuna_parameters = {
    "l2_gamma": LogUniformDistribution(low=1E-5, high=1E1),
    #"minimum_acceptable_return": UniformDistribution(low=0,high=0.1)
}

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    optuna_cv = OptunaSearchCV(
        estimator=MinimumVolatility(frequency=1),
        param_distributions=optuna_parameters,
        cv=KFold(10),
        n_jobs=16,
        n_trials=32,
        scoring=all_scorers['sharpe_ratio'],
        verbose=False
    ).fit(dow_train)

In [ ]:
optuna_cv.best_estimator_

In [ ]:
fig, ax = plt.subplots(figsize=(15,4))
equity_curve(MinimumVolatility().fit(dow_train).predict(dow_train)).plot(ax=ax)
equity_curve(optuna_cv.best_estimator_.predict(dow_train)).plot(ax=ax)
plt.legend()
plt.grid()

In [ ]:
backtest(
    dow_train,
    [
        MinimumVolatility().fit(dow_train).weights_,
         optuna_cv.best_estimator_.weights_
    ],
    frequency=1
)